<a href="https://colab.research.google.com/github/Pedro0504/demo-repo/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TextVectorization
import seaborn as sns
import matplotlib.pyplot as plt


print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
#First we have to load the data
train_dataset = pd.read_csv(train_file_path, delimiter='\t')
test_dataset = pd.read_csv(test_file_path, delimiter='\t')
# the names of the columns are long so it's necessary to change them
train_dataset.rename(columns ={'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.': 'sms', 'ham':'label'}, inplace = True)
test_dataset.rename(columns ={'i am in hospital da. . i will return home in evening': 'sms', 'ham':'label'}, inplace = True)
# the data in the class column needs to be numerical
# so we use the map() to change the values
train_dataset['label'] = train_dataset['label'].map({'ham':0, 'spam':1})
test_dataset['label'] = test_dataset['label'].map({'ham':0, 'spam':1})
# the data then is divided in training and testing
# data and labels
train_label = train_dataset.pop('label').astype('float32')
test_label = test_dataset.pop('label').astype('float32')
train_data = train_dataset['sms']
test_data = test_dataset['sms']




In [ ]:
# we need to create a TextVectorization() object
# and adapting it to the train_data
vector = tf.keras.layers.TextVectorization()
vector.adapt(train_data)

In [ ]:
# now we can create the model
# Create Sequential model, as in tutorial,
# simple combination of Embedding, Dense, Bidirectional and LSTM

from tensorflow.keras.layers import Embedding, Dense, Bidirectional, LSTM
model = tf.keras.Sequential([
    Embedding(
        input_dim = len(vector.get_vocabulary()),
        output_dim = 64,
        mask_zero = True),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
    ])

#model compile
model.compile(
    loss = 'binary_crossentropy',
    optimizer = tf.keras.optimizers.Adam(1e-4),
    metrics = ['accuracy']
    )
model.summary()

In [ ]:
# we make the compilation,
# we create a early stop variable to control the validation loss
early_stop = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 6, restore_best_weights = True)
history = model.fit(
    vector(train_data),train_label,
    epochs = 10,
    validation_data = (vector(test_data), test_label),
    validation_steps = 30,
    callbacks = [early_stop])


In [ ]:
# check for the model behavior during training
loss, accuracy = model.evaluate(vector(test_data), test_label)
print(f'Loss:{loss}')
print(f'Accuracy:{accuracy}')
import seaborn as sns
sns.lineplot(x = history.epoch, y = history.history['accuracy'], label = 'ACC')
sns.lineplot(x = history.epoch, y = history.history['val_accuracy'], label = 'VAL_ACC')

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_vect = vector(tf.expand_dims(pred_text, 0))
  pred = model.predict(pred_vect)
  if (pred[0]) > 0.5:
    prediction = 'spam'
  else:
    prediction = 'ham'
  return (pred[0],prediction)


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
